In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
# Started with 5 transit properties: period, time0bk, impact, duration, depth; scored .505/.487
# Added after first run: prad, teq, insol; scored .312/.316 - got worse
# Added TCE (threshold-crossing events) columns: model_snr, tce_plnt_num and Stellar params: steff, slogg, srad; scored .279/.290
# Removed 'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration', 'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol'
# Using highest correlations from the corr matrix:
selected_features = df[['koi_prad', 'koi_impact', 'koi_time0bk', 'koi_model_snr', 'koi_depth', 'koi_fpflag_ss', 'koi_srad', 'koi_insol', 'koi_slogg', 'koi_kepmag']]
selected_features.head()

,koi_prad,koi_impact,koi_time0bk,koi_model_snr,koi_depth,koi_fpflag_ss,koi_srad,koi_insol,koi_slogg,koi_kepmag
0,2.83,0.586,162.513840,25.8,874.8,0,0.927,9.11,4.467,15.347
1,14.60,0.969,175.850252,76.3,10829.0,1,0.868,39.30,4.544,15.436
2,33.46,1.276,170.307565,505.6,8079.2,1,0.791,891.96,4.564,15.597
3,2.75,0.701,171.595550,40.9,603.3,0,1.046,926.16,4.438,15.509
4,2.77,0.762,172.979370,40.2,686.0,0,0.972,427.65,4.486,15.714


# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split

X = selected_features
y = df['koi_disposition']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [5]:
X_train.head()

,koi_prad,koi_impact,koi_time0bk,koi_model_snr,koi_depth,koi_fpflag_ss,koi_srad,koi_insol,koi_slogg,koi_kepmag
6122,1.24,0.150,133.077240,10.8,123.1,0,1.125,253.30,4.327,14.725
6370,0.86,0.291,132.020050,13.8,114.6,1,0.797,2891.64,4.578,15.770
2879,3.21,0.970,134.460380,254.3,641.1,0,0.963,226.81,4.481,13.099
107,2.25,0.300,174.662240,38.4,875.4,0,0.779,55.37,4.536,15.660
29,12.21,0.831,172.258529,696.5,9802.0,0,1.082,349.40,4.359,15.263


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [6]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Train the Model



In [7]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model

LogisticRegression()

In [8]:
log_model.fit(X_train, y_train)

LogisticRegression()

In [9]:
print(f"Training Data Score: {log_model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {log_model.score(X_test_scaled, y_test)}")

Training Data Score: 0.2530993705893572
Testing Data Score: 0.2797482837528604


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [10]:
# Create the GridSearchCV model
from sklearn.svm import SVC
svc_model = SVC(kernel='linear')
svc_model

SVC(kernel='linear')

In [11]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[1,5,10,50], 'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(svc_model, param_grid, verbose=3)

In [12]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] ................... C=1, gamma=0.0001, score=0.565, total=   0.5s
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[CV] ................... C=1, gamma=0.0001, score=0.551, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
[CV] ................... C=1, gamma=0.0001, score=0.562, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.573, total=   0.4s
[CV] C=1, gamma=0.0001 ...............................................
[CV] ................... C=1, gamma=0.0001, score=0.544, to

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [13]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.5672321513036771


# Save the Model

In [14]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'log_grid.sav'
joblib.dump(svc_model, filename)

['log_grid.sav']